<a href="https://colab.research.google.com/github/bowon01/dacon/blob/main/%EB%8D%B0%EC%9D%B4%EC%BD%98_%EC%A0%91%EC%95%88%EC%8B%9C%EA%B0%84_%EC%98%88%EC%B8%A1(%EB%9E%9C%EB%8D%A4%ED%8F%AC%EB%A0%88%EC%8A%A4%ED%8A%B8).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 데이콘

- 제공된 데이터
    - train.csv
    - test.csv
- 예측할 컬럼 : 접안시간




In [ ]:
from google.colab import drive
drive.mount('/content/drive')

KeyboardInterrupt: ignored

## 데이터 불러오기

In [ ]:
import pandas as pd
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

## EDA

In [ ]:
# 크기 확인
train.shape, test.shape

((1490, 10), (497, 9))

In [ ]:
# 샘플 확인
train.head()

,Unnamed: 0,Age,Employment Type,GraduateOrNot,AnnualIncome,FamilyMembers,ChronicDiseases,FrequentFlyer,EverTravelledAbroad,TravelInsurance
0,888,28,Private Sector/Self Employed,Yes,1250000,6,1,No,No,0
1,1308,31,Private Sector/Self Employed,Yes,1250000,7,1,No,No,0
2,151,29,Private Sector/Self Employed,Yes,1200000,7,0,No,No,1
3,396,33,Government Sector,Yes,650000,6,1,No,No,1
4,135,28,Private Sector/Self Employed,Yes,800000,6,0,No,Yes,1


In [ ]:
# 타입 확인
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [ ]:
# 결측치 확인 train
train.isnull().sum()

Unnamed: 0             0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
TravelInsurance        0
dtype: int64

In [ ]:
# 결측치 확인 test
test.isnull().sum()

Unnamed: 0             0
Age                    0
Employment Type        0
GraduateOrNot          0
AnnualIncome           0
FamilyMembers          0
ChronicDiseases        0
FrequentFlyer          0
EverTravelledAbroad    0
dtype: int64

In [ ]:
# 기초 통계 획인
train.describe()

,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
count,1490.000000,1490.000000,1.490000e+03,1490.000000,1490.000000,1490.000000
mean,992.277181,29.600000,9.310738e+05,4.755705,0.280537,0.352349
std,566.637006,2.887829,3.763057e+05,1.603613,0.449412,0.477862
min,0.000000,25.000000,3.000000e+05,2.000000,0.000000,0.000000
25%,502.250000,28.000000,6.000000e+05,4.000000,0.000000,0.000000
50%,994.500000,29.000000,9.000000e+05,5.000000,0.000000,0.000000
75%,1479.750000,32.000000,1.250000e+06,6.000000,1.000000,1.000000
max,1983.000000,35.000000,1.800000e+06,9.000000,1.000000,1.000000


In [ ]:
# 기초 통계 획인 object
train.describe(include='object')

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
count,1490,1490,1490,1490
unique,2,2,2,2
top,Private Sector/Self Employed,Yes,No,No
freq,1056,1270,1175,1209


In [ ]:
# 타겟(레이블)
train['TravelInsurance'].value_counts()

0    965
1    525
Name: TravelInsurance, dtype: int64

## 데이터 전처리 및 피처엔지니어링

In [ ]:
# 수치형 데이터와 범주형 데이터 분리
n_train = train.select_dtypes(exclude='object').copy()
c_train = train.select_dtypes(include='object').copy()
n_test = test.select_dtypes(exclude='object').copy()
c_test = test.select_dtypes(include='object').copy()

In [ ]:
# 수치형 변수
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
cols = ['Age', 'AnnualIncome', 'FamilyMembers', 'ChronicDiseases']
display(n_train.head())
n_train[cols] = scaler.fit_transform(n_train[cols])
n_test[cols] = scaler.transform(n_test[cols])
n_train.head()

,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
0,888,28,1250000,6,1,0
1,1308,31,1250000,7,1,0
2,151,29,1200000,7,0,1
3,396,33,650000,6,1,1
4,135,28,800000,6,0,1


,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance
0,888,-0.25,0.538462,0.5,1.0,0
1,1308,0.50,0.538462,1.0,1.0,0
2,151,0.00,0.461538,1.0,0.0,1
3,396,1.00,-0.384615,0.5,1.0,1
4,135,-0.25,-0.153846,0.5,0.0,1


In [ ]:
# 범주형 변수
display(c_train.head())
c_train = pd.get_dummies(c_train)
c_test = pd.get_dummies(c_test)
c_train.head()

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
0,Private Sector/Self Employed,Yes,No,No
1,Private Sector/Self Employed,Yes,No,No
2,Private Sector/Self Employed,Yes,No,No
3,Government Sector,Yes,No,No
4,Private Sector/Self Employed,Yes,No,Yes


,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes
0,0,1,0,1,1,0,1,0
1,0,1,0,1,1,0,1,0
2,0,1,0,1,1,0,1,0
3,1,0,0,1,1,0,1,0
4,0,1,0,1,1,0,0,1


In [ ]:
# 분리한 데이터 다시 합침
train = pd.concat([n_train, c_train], axis=1)
test = pd.concat([n_test, c_test], axis=1)
print(train.shape, test.shape)
train.head()

(1490, 14) (497, 13)


,Unnamed: 0,Age,AnnualIncome,FamilyMembers,ChronicDiseases,TravelInsurance,Employment Type_Government Sector,Employment Type_Private Sector/Self Employed,GraduateOrNot_No,GraduateOrNot_Yes,FrequentFlyer_No,FrequentFlyer_Yes,EverTravelledAbroad_No,EverTravelledAbroad_Yes
0,888,-0.25,0.538462,0.5,1.0,0,0,1,0,1,1,0,1,0
1,1308,0.50,0.538462,1.0,1.0,0,0,1,0,1,1,0,1,0
2,151,0.00,0.461538,1.0,0.0,1,0,1,0,1,1,0,1,0
3,396,1.00,-0.384615,0.5,1.0,1,1,0,0,1,1,0,1,0
4,135,-0.25,-0.153846,0.5,0.0,1,0,1,0,1,1,0,0,1


## 검증데이터 분리

In [ ]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train.drop('TravelInsurance', axis=1),
                                            train['TravelInsurance'],
                                            test_size=0.1,
                                            random_state=1204)
X_tr.shape, X_val.shape, y_tr.shape, y_val.shape

((1341, 13), (149, 13), (1341,), (149,))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=400, max_depth=9, random_state=1204)
rf.fit(X_tr, y_tr)
pred = rf.predict_proba(X_val)[:,1]

In [ ]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_val, pred)

0.8277551020408163

## 예측

In [ ]:
# test 예측
pred = rf.predict_proba(test)[:,1]

In [ ]:
# csv 파일 생성 (예시와 다른 형태)
pd.DataFrame({'index':test.index,'y_pred':pred}).to_csv('0000.csv', index=False)

In [ ]:
# 또는
pd.DataFrame({'y_pred':pred}).reset_index().to_csv('1111.csv', index=False)

In [ ]:
# 확인
pd.read_csv("0000.csv")

,index,y_pred
0,0,0.149264
1,1,0.128597
2,2,0.125320
3,3,0.985559
4,4,0.153028
...,...,...
492,492,0.104853
493,493,0.983758
494,494,0.121372
495,495,0.965155


In [ ]:
# 확인
pd.read_csv("1111.csv")

,index,y_pred
0,0,0.149264
1,1,0.128597
2,2,0.125320
3,3,0.985559
4,4,0.153028
...,...,...
492,492,0.104853
493,493,0.983758
494,494,0.121372
495,495,0.965155


## 체점
- 수험자는 알 수 없는 부분임

In [ ]:
y_test = pd.read_csv("y_test.csv")
roc_auc_score(y_test, pred)

0.7875693000693

In [ ]:
# 당신의 score는?: 0.7875693000693